# ML0bag Vision System - Training Notebook

**Luxury Bag Defect Detection & Authentication using YOLOv11**

This notebook runs the full training pipeline. You can use it in:
- **Google Colab** (with GPU runtime or connected to your local RTX 5090)
- **Jupyter** running locally on your machine

---

## Setup Options

| Option | How | Best For |
|--------|-----|----------|
| **A) Colab + Local GPU** | Colab connects to Jupyter on your PC | Use your RTX 5090 with Colab's UI |
| **B) Colab GPU** | Use Colab's free/paid GPU | No local GPU available |
| **C) Local Jupyter** | Run Jupyter directly on your PC | Full local control |

## Step 0: Connect Colab to Your Local RTX 5090

**Skip this cell if running locally or using Colab's GPU.**

To use your RTX 5090 from Google Colab, you need to run a local Jupyter runtime:

### On your PC (one-time setup):

```bash
# 1. Install Jupyter with Colab local runtime support
pip install jupyter jupyter_http_over_ws
jupyter serverextension enable --py jupyter_http_over_ws

# 2. Start the local runtime (run this every time)
jupyter notebook \
  --NotebookApp.allow_origin='https://colab.research.google.com' \
  --port=8888 \
  --NotebookApp.port_retries=0
```

### In Google Colab:
1. Click the dropdown arrow next to **"Connect"** (top right)
2. Select **"Connect to a local runtime"**
3. Enter the URL shown in your terminal (e.g., `http://localhost:8888/?token=...`)
4. Click **Connect**

Now Colab runs on your RTX 5090!

---
## Step 1: Install Dependencies & Clone Project

In [ ]:
# Check if running in Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # Clone the project repo (update URL to your repo)
    !git clone https://github.com/YOUR_USERNAME/ML0bag-visionsystem.git
    %cd ML0bag-visionsystem
else:
    # If running locally, make sure you're in the project root
    import os
    # Adjust this path to where your project lives
    # os.chdir('/path/to/ML0bag-visionsystem')
    print(f"Working directory: {os.getcwd()}")

In [ ]:
# Install dependencies
!pip install -q ultralytics>=8.3.0 \
    torch torchvision \
    opencv-python-headless \
    huggingface-hub datasets \
    roboflow \
    Pillow numpy pandas pyyaml \
    tqdm matplotlib seaborn \
    loguru click albumentations

In [ ]:
# Verify GPU is available
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    props = torch.cuda.get_device_properties(0)
    # total_memory works across all PyTorch versions (total_mem was removed in 2.x)
    vram = getattr(props, 'total_memory', None) or getattr(props, 'total_mem', 0)
    print(f"VRAM: {vram / 1e9:.1f} GB")
else:
    print("WARNING: No GPU detected. Training will be very slow on CPU.")
    print("If using Colab: Runtime > Change runtime type > GPU")
    print("If local: Make sure NVIDIA drivers and CUDA are installed.")

In [ ]:
# Add project root to Python path
import os
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd()

# Handle nested clone: ML0bag-visionsystem/ML0bag-visionsystem/
# Walk down into nested copies until we find the deepest one with src/
while (PROJECT_ROOT / 'ML0bag-visionsystem' / 'src').exists():
    PROJECT_ROOT = PROJECT_ROOT / 'ML0bag-visionsystem'

# If src/ is not in current dir, check common locations
if not (PROJECT_ROOT / 'src').exists():
    for candidate in [
        Path('/home/programming/ML0bag-visionsystem/ML0bag-visionsystem'),
        Path('/home/programming/ML0bag-visionsystem'),
        Path('/content/ML0bag-visionsystem'),
        Path.home() / 'ML0bag-visionsystem',
    ]:
        if (candidate / 'src').exists():
            PROJECT_ROOT = candidate
            break

os.chdir(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"Project root: {PROJECT_ROOT}")
assert (PROJECT_ROOT / 'src').exists(), f"ERROR: src/ not found in {PROJECT_ROOT}"

# Verify imports work
from src.config.settings import ensure_dirs, get_device, load_config
from src.utils.logging import setup_logger

ensure_dirs()
print(f"Device: {get_device()}")
print("Setup complete!")

---
## Step 2: Download Datasets

You need API keys for some datasets:
- **Roboflow**: Free account at roboflow.com > Settings > API Key
- **HuggingFace**: Optional, for gated datasets > huggingface.co > Settings > Access Tokens
- **Kaggle**: For leather defects > kaggle.com > Settings > API > Create Token

In [ ]:
import os

# Set your API keys here (or use Colab Secrets)
# In Colab: click the key icon on the left sidebar to add secrets

if IN_COLAB:
    from google.colab import userdata
    try:
        os.environ['ROBOFLOW_API_KEY'] = userdata.get('ROBOFLOW_API_KEY')
    except Exception:
        pass
    try:
        os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
    except Exception:
        pass

# Or set them directly (don't commit this!):
# os.environ['ROBOFLOW_API_KEY'] = 'your_key_here'
# os.environ['HF_TOKEN'] = 'your_token_here'

In [ ]:
# Download Dataset 1: CVandDL Bag Logos (Roboflow) - Phase 1
from src.data.download import download_from_roboflow
from src.config.settings import DATA_DIR

roboflow_key = os.environ.get('ROBOFLOW_API_KEY', '')
if roboflow_key:
    logo_dir = download_from_roboflow(
        workspace='cvanddl',
        project='detection-bag-logo',
        version=1,
        api_key=roboflow_key,
        output_dir=DATA_DIR / 'raw' / 'roboflow_logos',
        format='yolov8',
    )
    print(f"Logos downloaded to: {logo_dir}")
else:
    print("Set ROBOFLOW_API_KEY to download the logo dataset.")
    print("Get a free key at: https://app.roboflow.com/settings/api")

In [ ]:
# Download Dataset 2: LFFD (HuggingFace) - Phase 3 Authentication
# Uses huggingface_hub directly (the datasets library has compatibility issues with this dataset)
from huggingface_hub import snapshot_download
from pathlib import Path

lffd_dir = Path('data/raw/lffd')
lffd_dir.mkdir(parents=True, exist_ok=True)

if not any(lffd_dir.rglob('*.jpg')) and not any(lffd_dir.rglob('*.parquet')):
    print("Downloading LFFD dataset (2 GB, may take a few minutes)...")
    snapshot_download(
        repo_id='Innovatiana/innv-luxury-fashion-dataset-fraud-detection',
        repo_type='dataset',
        local_dir=str(lffd_dir),
        token=os.environ.get('HF_TOKEN'),
    )
    print(f"LFFD downloaded to: {lffd_dir}")
else:
    print(f"LFFD already downloaded at: {lffd_dir}")

In [ ]:
# Download Dataset 3: Leather Defects (Kaggle) - Phase 2 supplement
import os
from pathlib import Path

os.environ['KAGGLE_USERNAME'] = 'marcosiced'
os.environ['KAGGLE_KEY'] = '5ecd78ca00c5ed99f788f36523beb29f'

leather_dir = Path('data/raw/leather_defects')

if leather_dir.exists() and any(d.is_dir() for d in leather_dir.iterdir()):
    print(f"Leather defects already at: {leather_dir}")
    for item in sorted(leather_dir.iterdir()):
        if item.is_dir():
            count = len(list(item.rglob('*.*')))
            print(f"  {item.name}/  ({count} files)")
else:
    !pip install -q kaggle
    leather_dir.mkdir(parents=True, exist_ok=True)
    !kaggle datasets download -d praveen2084/leather-defect-classification -p data/raw/leather_defects --unzip
    print(f"\nDownloaded to: {leather_dir}")
    for item in sorted(leather_dir.iterdir()):
        if item.is_dir():
            count = len(list(item.rglob('*.*')))
            print(f"  {item.name}/  ({count} files)")

---
## Step 3: Preprocess Datasets into YOLO Format

In [ ]:
import os
import zipfile
from pathlib import Path
from src.config.settings import DATA_DIR

# --- Extract LFFD zip if needed ---
lffd_zip = DATA_DIR / 'raw' / 'lffd' / 'dataset-chanel-luxury-bags.zip'
lffd_dir = DATA_DIR / 'raw' / 'lffd'

if lffd_zip.exists():
    # Count images already extracted
    existing_imgs = list(lffd_dir.rglob('*.jpg')) + list(lffd_dir.rglob('*.jpeg')) + list(lffd_dir.rglob('*.png'))
    # Only extract if we don't have many images yet (zip + readme = not extracted)
    if len(existing_imgs) < 100:
        print("Extracting LFFD zip (this may take a minute)...")
        with zipfile.ZipFile(lffd_zip, 'r') as zf:
            zf.extractall(lffd_dir)
        existing_imgs = list(lffd_dir.rglob('*.jpg')) + list(lffd_dir.rglob('*.jpeg')) + list(lffd_dir.rglob('*.png'))
        print(f"Extracted {len(existing_imgs)} images")
    else:
        print(f"LFFD already extracted: {len(existing_imgs)} images")

# --- Check Roboflow structure ---
rf_dir = DATA_DIR / 'raw' / 'roboflow_logos'
print("\n=== Roboflow Logos ===")
for split in ['train', 'valid', 'test']:
    img_dir = rf_dir / split / 'images'
    if img_dir.exists():
        count = len(list(img_dir.iterdir()))
        print(f"  {split}: {count} images")
    elif (rf_dir / split).exists():
        # Images might be directly in the split folder
        imgs = [f for f in (rf_dir / split).iterdir() if f.suffix.lower() in {'.jpg', '.jpeg', '.png'}]
        print(f"  {split}: {len(imgs)} images (flat structure)")

# --- Now preprocess all phases with force ---
print("\n" + "=" * 50)
print("Preparing Phase 1: Logo Detection")
print("=" * 50)
from src.data.preprocess import prepare_phase1, prepare_phase2, prepare_phase3

phase1_yaml = prepare_phase1(force=True)
print(f"Phase 1 data config: {phase1_yaml}\n")

print("=" * 50)
print("Preparing Phase 2: Condition Assessment")
print("=" * 50)
phase2_yaml = prepare_phase2(force=True)
print(f"Phase 2 data config: {phase2_yaml}\n")

print("=" * 50)
print("Preparing Phase 3: Authentication")
print("=" * 50)
phase3_dir = prepare_phase3(force=True)
print(f"Phase 3 data dir: {phase3_dir}")

In [ ]:
# Verify dataset sizes
from pathlib import Path
from src.config.settings import DATA_DIR

for phase in ['phase1', 'phase2', 'phase3']:
    phase_dir = DATA_DIR / 'processed' / phase
    print(f"\n--- {phase} ---")
    for split in ['train', 'val', 'test']:
        img_dir = phase_dir / 'images' / split
        if img_dir.exists():
            count = len(list(img_dir.iterdir()))
            print(f"  {split}: {count} images")
        else:
            # Check classification format (phase3)
            cls_dir = phase_dir / split
            if cls_dir.exists():
                for cls in cls_dir.iterdir():
                    if cls.is_dir():
                        count = len(list(cls.iterdir()))
                        print(f"  {split}/{cls.name}: {count} images")

---
## Step 4: Train Phase 1 - Bag & Logo Detection

Detects bags in the image and identifies the brand logo.

**Model**: YOLOv11m (medium) | **Expected time**: ~1-2 hours on RTX 5090

In [ ]:
from ultralytics import YOLO
from src.config.settings import load_config, get_device, MODELS_DIR, RUNS_DIR
import shutil

# Load phase 1 config
config = load_config('phase1_detection')
device = get_device()
print(f"Training on: {device}")

# Load pretrained YOLOv11 medium
model = YOLO(config['model']['architecture'])

# Resolve data path
data_path = str(PROJECT_ROOT / config['data']['config'])
print(f"Data config: {data_path}")

In [ ]:
# Train Phase 1
# workers=0 and pin_memory=False prevent CUDA pin memory errors
# when running through Colab local runtime
results = model.train(
    data=data_path,
    epochs=config['training']['epochs'],
    imgsz=config['data']['imgsz'],
    batch=config['training']['batch'],
    device=device,
    project=str(PROJECT_ROOT / config['output']['project']),
    name=config['output']['name'],
    patience=config['training']['patience'],
    optimizer=config['training']['optimizer'],
    lr0=config['training']['lr0'],
    cos_lr=config['training']['cos_lr'],
    save_period=config['output']['save_period'],
    plots=True,
    verbose=True,
    workers=0,
    **config.get('augmentation', {}),
)

In [ ]:
# Save best weights
best_src = Path(results.save_dir) / 'weights' / 'best.pt'
best_dst = MODELS_DIR / 'phase1' / 'best.pt'
best_dst.parent.mkdir(parents=True, exist_ok=True)

if best_src.exists():
    shutil.copy2(best_src, best_dst)
    print(f"Phase 1 best model saved to: {best_dst}")

# Show training results
from IPython.display import Image, display
results_img = Path(results.save_dir) / 'results.png'
if results_img.exists():
    display(Image(filename=str(results_img), width=900))

---
## Step 5: Train Phase 2 - Condition / Defect Detection

Detects defects: scratches, tears, stains, wear, deformation.

**Model**: YOLOv11m | **Expected time**: ~2-4 hours on RTX 5090

In [ ]:
# Load phase 2 config
config2 = load_config('phase2_condition')

model2 = YOLO(config2['model']['architecture'])
data_path2 = str(PROJECT_ROOT / config2['data']['config'])

results2 = model2.train(
    data=data_path2,
    epochs=config2['training']['epochs'],
    imgsz=config2['data']['imgsz'],
    batch=config2['training']['batch'],
    device=device,
    project=str(PROJECT_ROOT / config2['output']['project']),
    name=config2['output']['name'],
    patience=config2['training']['patience'],
    optimizer=config2['training']['optimizer'],
    lr0=config2['training']['lr0'],
    cos_lr=config2['training']['cos_lr'],
    save_period=config2['output']['save_period'],
    plots=True,
    verbose=True,
    workers=0,
    **config2.get('augmentation', {}),
)

In [ ]:
# Save Phase 2 best weights
best_src2 = Path(results2.save_dir) / 'weights' / 'best.pt'
best_dst2 = MODELS_DIR / 'phase2' / 'best.pt'
best_dst2.parent.mkdir(parents=True, exist_ok=True)

if best_src2.exists():
    shutil.copy2(best_src2, best_dst2)
    print(f"Phase 2 best model saved to: {best_dst2}")

results_img2 = Path(results2.save_dir) / 'results.png'
if results_img2.exists():
    display(Image(filename=str(results_img2), width=900))

---
## Step 6: Train Phase 3 - Authentication (Real vs Fake)

Classifies bags as authentic or counterfeit.

**Model**: YOLOv11s-cls (classification) | **Expected time**: ~30-60 min on RTX 5090

**IMPORTANT**: Phase 3 requires manually curated data. The LFFD dataset images are
initially all placed in the `authentic` folder. You must:
1. Review images and move counterfeit ones to `data/processed/phase3/train/counterfeit/`
2. Also populate `data/processed/phase3/val/counterfeit/`
3. Aim for at least 500+ images per class for reasonable results

In [ ]:
# Check Phase 3 data readiness
phase3_train = DATA_DIR / 'processed' / 'phase3' / 'train'
phase3_val = DATA_DIR / 'processed' / 'phase3' / 'val'

auth_count = len(list((phase3_train / 'authentic').glob('*'))) if (phase3_train / 'authentic').exists() else 0
fake_count = len(list((phase3_train / 'counterfeit').glob('*'))) if (phase3_train / 'counterfeit').exists() else 0

print(f"Phase 3 training data:")
print(f"  Authentic: {auth_count} images")
print(f"  Counterfeit: {fake_count} images")

if fake_count < 100:
    print(f"\nWARNING: Not enough counterfeit samples ({fake_count}).")
    print("You need to manually sort counterfeit images before training Phase 3.")
    print(f"Place them in: {phase3_train / 'counterfeit'}/")
    PHASE3_READY = False
else:
    PHASE3_READY = True
    print("\nPhase 3 data looks ready!")

In [ ]:
# Train Phase 3 (only if data is ready)
if PHASE3_READY:
    config3 = load_config('phase3_authentication')
    
    model3 = YOLO(config3['model']['architecture'])
    data_path3 = str(PROJECT_ROOT / config3['data']['config'])
    
    results3 = model3.train(
        data=data_path3,
        epochs=config3['training']['epochs'],
        imgsz=config3['data']['imgsz'],
        batch=config3['training']['batch'],
        device=device,
        project=str(PROJECT_ROOT / config3['output']['project']),
        name=config3['output']['name'],
        patience=config3['training']['patience'],
        optimizer=config3['training']['optimizer'],
        lr0=config3['training']['lr0'],
        cos_lr=config3['training']['cos_lr'],
        save_period=config3['output']['save_period'],
        plots=True,
        verbose=True,
        workers=0,
        **config3.get('augmentation', {}),
    )
    
    # Save weights
    best_src3 = Path(results3.save_dir) / 'weights' / 'best.pt'
    best_dst3 = MODELS_DIR / 'phase3' / 'best.pt'
    best_dst3.parent.mkdir(parents=True, exist_ok=True)
    if best_src3.exists():
        shutil.copy2(best_src3, best_dst3)
        print(f"Phase 3 best model saved to: {best_dst3}")
else:
    print("Skipping Phase 3 training - not enough counterfeit samples.")
    print("Complete the annotation step above first.")

---
## Step 7: Test the Full Cascade Pipeline

Run all 3 models in sequence: Detect bag -> Check condition -> Authenticate

In [ ]:
# Test with a single image
import cv2
import matplotlib.pyplot as plt
from src.inference.pipeline import CascadePipeline

# Initialize the cascade pipeline (loads all available trained models)
pipeline = CascadePipeline()

# Load a test image - replace with your own image path
test_image_path = 'path/to/your/test/bag/image.jpg'  # <-- CHANGE THIS

if Path(test_image_path).exists():
    image = cv2.imread(test_image_path)
    
    # Run full pipeline
    results = pipeline.process(image)
    
    # Print results
    for i, r in enumerate(results):
        print(f"\nBag #{i+1}:")
        print(f"  Brand/Type: {r.bag_type} ({r.detection_conf:.1%})")
        if r.condition:
            print(f"  Condition: {r.condition} ({r.condition_conf:.1%})")
        if r.auth_result:
            print(f"  Authentication: {r.auth_result} ({r.auth_conf:.1%})")
    
    # Show annotated image
    annotated = pipeline.annotate(image, results)
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title('Cascade Pipeline Results')
    plt.show()
else:
    print(f"Test image not found: {test_image_path}")
    print("Upload a bag image and update the path above.")

In [ ]:
# Quick test: upload an image in Colab
if IN_COLAB:
    from google.colab import files
    print("Upload a bag image to test:")
    uploaded = files.upload()
    
    for filename in uploaded:
        image = cv2.imread(filename)
        results = pipeline.process(image)
        
        for i, r in enumerate(results):
            print(f"\nBag #{i+1}: {r.bag_type} ({r.detection_conf:.1%})")
            if r.condition:
                print(f"  Condition: {r.condition} ({r.condition_conf:.1%})")
            if r.auth_result:
                print(f"  Auth: {r.auth_result} ({r.auth_conf:.1%})")
        
        annotated = pipeline.annotate(image, results)
        plt.figure(figsize=(12, 8))
        plt.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()

---
## Step 8: Export Models for Production

Export to ONNX for faster inference or deployment.

In [ ]:
# Export trained models to ONNX format
for phase in ['phase1', 'phase2', 'phase3']:
    model_path = MODELS_DIR / phase / 'best.pt'
    if model_path.exists():
        print(f"Exporting {phase}...")
        m = YOLO(str(model_path))
        m.export(format='onnx')
        print(f"  Exported to: {MODELS_DIR / phase / 'best.onnx'}")
    else:
        print(f"  {phase}: No trained model found, skipping.")

---
## Download Trained Models (Colab only)

If training in Colab, download your models to keep them.

In [ ]:
if IN_COLAB:
    import shutil
    # Zip all models
    shutil.make_archive('trained_models', 'zip', str(MODELS_DIR))
    files.download('trained_models.zip')
    print("Models downloaded!")
else:
    print(f"Models are saved at: {MODELS_DIR}")